In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr

In [50]:
df=pd.read_csv("./data/tourism/KPI_touristique.csv")
trands=pd.read_csv("trends.csv")

In [51]:
def filter_tourism_keywords(keywords):
    """Garde uniquement les mots-clés touristiques pertinents"""
    import re
    
    # Mots-clés à EXCLURE (non-touristiques)
    EXCLUDE = [
        r'\b(oil|argan|beauty|cosmetic|shampoo|body|hair)\b',
        r'\b(recipe|food|chilli|chocolate|cake|cooking)\b',
        r'\b(bbc|news|article|blog|instagram|facebook)\b',
        r'\b(price|buy|shop|store|amazon|ebay)\b',
        r'\b(job|work|salary|visa application)\b',
    ]
    
    # Mots-clés à INCLURE (touristiques)
    INCLUDE = [
        r'\b(hotel|riad|resort|accommodation|stay|booking)\b',
        r'\b(flight|airport|airline|ticket|travel)\b',
        r'\b(tour|excursion|visit|guide|itinerary|trip)\b',
        r'\b(marrakech|casablanca|fez|chefchaouen|essaouira|tangier|agadir)\b',
        r'\b(sahara|desert|atlas|medina|kasbah|souk)\b',
        r'\b(weather|temperature|climate|best time)\b',
        r'\b(things to do|places to visit|attractions)\b',
        r'\b(restaurant|cuisine|dining)\b',
        r'\b(beach|mountain|hiking|surfing|activities)\b',
    ]
    
    filtered = []
    for kw in keywords:
        kw_lower = kw.lower()
        
        # Exclure si non-touristique
        if any(re.search(pattern, kw_lower) for pattern in EXCLUDE):
            continue
        
        # Inclure si touristique
        if any(re.search(pattern, kw_lower) for pattern in INCLUDE):
            filtered.append(kw)
    
    return filtered

In [52]:
l=trands.columns.tolist()
l.remove("Date")

In [53]:
keywords_cleaned = filter_tourism_keywords(l)
len(keywords_cleaned)
trands=trands[["Date"] + keywords_cleaned]
data=pd.merge(trands, df[['Date','TES']], on="Date")

In [ ]:
l=data.columns.tolist()
l.remove("Date")
data=data[l]

### Calcul des corrélations avec décalages temporels


In [73]:
dataframe = data

target_col = 'TES'  
keyword_cols = [col for col in dataframe.columns if col != target_col]

results = []


for keyword in keyword_cols:
    correlations = {}
    
    # Tester différents lags (0 à 12 mois)
    for lag in range(1, 13):
    # Décaler la série du keyword
    
        keyword_lagged = dataframe[keyword].shift(lag)

        # Supprimer les valeurs manquantes
        valid_idx = ~(keyword_lagged.isna() | dataframe[target_col].isna())

        if valid_idx.sum() > 10:  # S'assurer qu'il y a assez de données
            # Calculer la corrélation de Pearson
            corr, p_value = pearsonr(
                keyword_lagged[valid_idx], 
                dataframe[target_col][valid_idx]
            )
            correlations[lag] = {
                'correlation': corr,
                'p_value': p_value
            }
    
    # Trouver le lag optimal (corrélation maximale)
    if correlations:
        best_lag = max(correlations.keys(), 
                      key=lambda x: abs(correlations[x]['correlation']))
        best_corr = correlations[best_lag]['correlation']
        best_pvalue = correlations[best_lag]['p_value']
        
        results.append({
            'keyword': keyword,
            'best_lag': best_lag,
            'correlation': best_corr,
            'p_value': best_pvalue,
            'all_correlations': correlations
        })

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('correlation', 
                                    key=abs, 
                                    ascending=False)

print(results_df[['keyword', 'best_lag', 'correlation', 'p_value']])

C:\Users\masta\AppData\Local\Temp\ipykernel_17164\94221215.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = pearsonr(


                               keyword  best_lag  correlation       p_value
204                  marrakech airport         1     0.789740  1.052126e-37
227                          marrakech         1     0.786069  3.835629e-37
39                      marrakech souk         1     0.767942  1.600169e-34
487                    marrakech souks         1     0.759295  2.353308e-33
217                    morocco airport         1     0.756005  6.354014e-33
..                                 ...       ...          ...           ...
298           morocco itinerary 4 days         8     0.054818  4.857016e-01
133  moroccan restaurant san francisco        12     0.053004  5.056253e-01
112        moroccan restaurant seattle        11    -0.052512  5.082489e-01
76       moroccan restaurant st albans        12     0.051779  5.155229e-01
198    spain morocco itinerary 10 days         8     0.040764  6.042773e-01

[490 rows x 4 columns]


### Sélection des variables finales

In [74]:
# Filtrer les keywords avec corrélation >= 70% (0.7)
threshold = 0.7
selected_keywords = results_df[
    abs(results_df['correlation']) >= threshold
].copy()

print(f"\nNombre de keywords sélectionnés avec corrélation >= 70% : {len(selected_keywords)}")
print("\nKeywords sélectionnés:")
print(selected_keywords[['keyword', 'best_lag', 'correlation']])

# Créer le dataset final avec les variables décalées
final_df = dataframe[[target_col]].copy()

for _, row in selected_keywords.iterrows():
    keyword = row['keyword']
    lag = row['best_lag']
    #final_df[f"{keyword}_lag{lag}"] = dataframe[keyword].shift(lag)
    final_df[f"{keyword}"] = dataframe[keyword].shift(lag)
# Supprimer les lignes avec valeurs manquantes
final_df = final_df.dropna()

#print(f"\nDataset final: {final_df.shape}")
final_df[selected_keywords['keyword'].head(3)].to_csv('dataset_final_avec_lags.csv', index=False)
print()
print("="*80)
print("\nTop 3 keywords sélectionnés:")
print(selected_keywords[['keyword', 'best_lag', 'correlation']].head(3))


Nombre de keywords sélectionnés avec corrélation >= 70% : 10

Keywords sélectionnés:
                        keyword  best_lag  correlation
204           marrakech airport         1     0.789740
227                   marrakech         1     0.786069
39               marrakech souk         1     0.767942
487             marrakech souks         1     0.759295
217             morocco airport         1     0.756005
408        aeropuerto marrakech         1     0.729697
188  best time to visit vietnam         1     0.728172
230                   essaouira         1     0.722959
456     marrakech to casablanca         1     0.712056
126  jardin majorelle marrakech         1     0.700372


Top 3 keywords sélectionnés:
               keyword  best_lag  correlation
204  marrakech airport         1     0.789740
227          marrakech         1     0.786069
39      marrakech souk         1     0.767942


In [75]:
matrice = data[["TES","marrakech airport","marrakech","marrakech souk"]].corr()
print(matrice)

                        TES  marrakech airport  marrakech  marrakech souk
TES                1.000000           0.795864   0.835605        0.783228
marrakech airport  0.795864           1.000000   0.845051        0.866740
marrakech          0.835605           0.845051   1.000000        0.872445
marrakech souk     0.783228           0.866740   0.872445        1.000000


In [127]:
mara=pd.read_csv("marakeche.csv")
mara["Date"]=pd.to_datetime(mara["Date"])
mara["Date"]=mara["Date"].dt.strftime("%m-%Y")
mara = mara[~mara["Date"].astype(str).str.contains("2020")]


In [128]:
da=pd.merge(mara,df,on="Date")
da["marrakech"]=da["marrakech"].shift(1)
da.rename(columns={"marrakech":"marrakech_lag_1"},inplace=True)

In [134]:
da

,Date,marrakech_lag_1,TES,MRE,Total Arrivées,Nuité,Taux d'occupation
0,01-2010,NaN,291608.0,216485.0,508093,NaN,NaN
1,02-2010,56.0,327017.0,214346.0,541363,NaN,NaN
2,03-2010,61.0,437338.0,234940.0,672278,NaN,NaN
3,04-2010,62.0,432338.0,232737.0,665075,NaN,NaN
4,05-2010,60.0,430974.0,241339.0,672313,NaN,NaN
...,...,...,...,...,...,...,...
169,02-2025,84.0,761377.0,634800.0,1396177,1797178.0,50.0
170,03-2025,84.0,768739.0,583272.0,1352011,1754640.0,40.0
171,04-2025,83.0,966724.0,698077.0,1664801,2463774.0,57.0
172,05-2025,86.0,NaN,NaN,154004,2305539.0,54.0


In [136]:
da.to_csv("data+marrakach.csv",index=False)